In [2]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir("/content/gdrive/MyDrive/DISSERTATION")
print(os.listdir())

Mounted at /content/gdrive/
['simple_hilbert.py', 'advection_block.py', '__pycache__', 'CAE_on_structured_mesh.ipynb', 'Flowpast', 'animation.mp4', 'MeshCNN', 'fpc.obj', 'cache', 'fpc_000.npz', 'fpc.edges', 'fpc_000.npz (Unzipped Files)', 'Advection_of_a_block_over_structured_mesh.ipynb', 'adobe.obj', 'fpc2.obj', 'Diss.ipynb', 'csv_data', 'fpc3.obj', 'animation2.mp4', 'Diss2.ipynb']


In [9]:
#Importing a bunch of stuff from MeshCNN
os.chdir("./MeshCNN")
import models
import options.train_options
os.chdir("./models")
import networks
os.chdir("./layers")
import mesh
import mesh_pool
os.chdir("../../../")

In [4]:
#Importing a bunch more libraries
import numpy as np
import matplotlib.pyplot as plt
import torch  # Pytorch
import torch.nn as nn  # Neural network module
import torch.nn.functional as fn  # Function module
from torchvision import datasets  # Datasets from torchvision
from torchvision import transforms  # Transforms from torchvision
from torch.utils.data import DataLoader
from matplotlib import animation

In [5]:
#Opt class from MeshCNN, to create a "Mesh" class you need an option class for data augmentation purposes, hard to get mesh loading to work otherwise
class opt:
  num_aug = 1

In [6]:
#Creating my flow past cylinder mesh
fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)

In [7]:
len(fpcmesh.vs)

3571

In [10]:
#Printing out the number of edges
print(fpcmesh.edges_count)

10421


In [11]:
#Printing out the number of unpoolable (without unionpool) edges 
edges = 0
for i in range(len(fpcmesh.gemm_edges)):
  if fpcmesh.gemm_edges[i][2] == -1 or fpcmesh.gemm_edges[i][3] == -1:
    edges+=1
print(edges)

292


In [12]:
#Loading velocities
vels = open("./csv_data/data_11.csv")
velocities = np.loadtxt(vels, delimiter=",")[:,3:5]

In [13]:
velocities[:10]

array([[ 4.56906740e-03,  1.43258235e-03],
       [ 4.95392011e-02, -1.51700762e-03],
       [-1.31928858e-03,  3.03326737e-05],
       [ 4.82493395e-02, -1.96715355e-03],
       [ 3.90000030e-02, -5.00809951e-08],
       [ 3.89999999e-02,  1.25708626e-11],
       [ 3.90000001e-02, -2.91025909e-11],
       [ 3.90000039e-02,  5.87965542e-08],
       [-8.66490183e-04,  1.23226801e-02],
       [-2.65637651e-03,  2.30785012e-02]])

In [14]:
print(np.shape(velocities))

(3571, 2)


In [15]:
print(fpcmesh.vs)

[[0.25       0.2        0.        ]
 [0.2        0.25       0.        ]
 [0.15       0.2        0.        ]
 ...
 [0.00612467 0.12981302 0.        ]
 [0.14262118 0.21670639 0.        ]
 [0.06215101 0.40337493 0.        ]]


In [16]:
print(fpcmesh.vs[:,1])

[0.2        0.25       0.2        ... 0.12981302 0.21670639 0.40337493]


In [17]:
vels = np.sqrt(velocities[:,0]**2 + velocities[:,1]**2)

In [18]:
print(vels)

[0.00478839 0.04956242 0.00131964 ... 0.0389988  0.02337547 0.03973314]


In [266]:
print(fpcmesh.edges)

[[ 109  383]
 [ 108  109]
 [ 197  582]
 ...
 [ 113  114]
 [ 112  113]
 [ 112 3570]]


In [20]:
EdgeFeatures = [[(velocities[edge[0]][0]+velocities[edge[1]][0])/2,(velocities[edge[0]][1]+velocities[edge[1]][1])/2] for edge in fpcmesh.edges]
velx = torch.Tensor(np.array([[EdgeFeatures[:,0],EdgeFeatures[:,1]]]))

In [21]:
print(EdgeFeatures)

[[0.03906411699498141, 2.8655243881378853e-05], [0.03903172179070892, 2.512233677213212e-05], [0.03904624215944391, 1.8059636286605244e-05], [0.03907156668973084, -3.4139256477077464e-05], [0.03905151103679454, -1.9229662898131896e-05], [0.03903526397462863, -2.856667958499999e-05], [0.03903398644847694, -5.398662728811485e-05], [0.03905257240806966, -9.476345093499368e-05], [0.03907390558715765, -8.42530968108244e-05], [0.03903520415672065, -0.00010737988632508299], [0.03907512329540136, -0.00013764635584779253], [0.03903168484789383, 4.965857646675223e-05], [0.03906747737245261, 7.944897598004773e-05], [0.03904956559410001, 9.338960807989423e-05], [0.03899999992412683, -4.819974286779219e-11], [0.038999999933537366, -6.988016371081949e-11], [0.03899999993467207, -5.952999653664874e-11], [0.039000058869335996, -7.031195177886631e-09], [0.03900006118399506, 2.3700390071950224e-09], [0.039000070921730114, -1.4042334498127114e-08], [0.03899992765849442, -7.255839922115512e-09], [0.038999

In [ ]:
velx = torch.Tensor(np.array([[EdgeFeatures[:,0],EdgeFeatures[:,1]]]))

In [22]:
EdgeFeatures = np.array(EdgeFeatures)

In [117]:
#This is an encoder with 0 pooling operations that has two convolutions from 1 to 3 channels and from 3 to 6 channels.
enc = networks.MeshEncoder([10421,7000],[2,2,2])

In [118]:
enc

MeshEncoder(
  (convs): ModuleList(
    (0): DownConv(
      (conv1): MeshConv(
        (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
      )
      (bn): ModuleList(
        (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (pool): MeshPool()
    )
    (1): DownConv(
      (conv1): MeshConv(
        (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
      )
      (bn): ModuleList(
        (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
    )
  )
)

In [179]:
up_convs = []

convs = [2,2,2]
unrolls = [10421]

for i in range(len(convs) - 2):
    if i < len(unrolls):
        unroll = unrolls[i]
    else:
        unroll = 0
    up_convs.append(networks.UpConv(convs[i], convs[i + 1], unroll=unroll))
final_conv = networks.UpConv(convs[-2], convs[-1], unroll=False, transfer_data=False)
up_convs = nn.ModuleList(up_convs)

In [180]:
print(up_convs)

ModuleList(
  (0): UpConv(
    (up_conv): MeshConv(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
    )
    (conv1): MeshConv(
      (conv): Conv2d(4, 2, kernel_size=(1, 5), stride=(1, 1))
    )
    (bn): ModuleList(
      (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    )
    (unroll): MeshUnpool()
  )
)


In [125]:
convs = [2,2,2]

In [126]:
convs[-2]

2

In [113]:
print(fpcmesh.edges_count)

10421


In [102]:
print(velx.size())

torch.Size([1, 2, 10421])


In [106]:
a,b = enc.forward([velx,[fpcmesh]])

In [107]:
print(a.size())

torch.Size([1, 2, 2000])


In [292]:
#Creating my flow past cylinder mesh
fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)

In [256]:
fpcmesh.init_history()

In [259]:
sys.getsizeof(fpcmesh)

64

In [257]:
encdek(velx,[fpcmesh])

tensor([[[0.1928, 0.0000, 0.4692,  ..., 0.4540, 0.5425, 0.5041],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<SqueezeBackward1>)

In [250]:
encdek = networks.MeshEncoderDecoder([10421,8000,6000,4000,2000],[2,2,2,2,2,2],[2,2,2,2,2,2],transfer_data = True)

In [251]:
encdek

MeshEncoderDecoder(
  (encoder): MeshEncoder(
    (convs): ModuleList(
      (0): DownConv(
        (conv1): MeshConv(
          (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
        )
        (bn): ModuleList(
          (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
        (pool): MeshPool()
      )
      (1): DownConv(
        (conv1): MeshConv(
          (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
        )
        (bn): ModuleList(
          (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
        (pool): MeshPool()
      )
      (2): DownConv(
        (conv1): MeshConv(
          (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1))
        )
        (bn): ModuleList(
          (0): InstanceNorm2d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
        (pool): MeshPool()
      )
      (3): DownConv(
        (conv1): MeshConv(

In [252]:
a = encdek.forward(velx,[fpcmesh])

In [253]:
print(a.size())

torch.Size([1, 2, 10421])


In [254]:
!pip install pycm livelossplot
%pylab inline
from livelossplot import PlotLosses

     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 573kB 15.6MB/s 
Populating the interactive namespace from numpy and matplotlib


In [263]:
import random 
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True

device = 'cuda'  # Set out device to GPU

print('Cuda installed, running on GPU!')  # print sentence

Cuda installed, running on GPU!


In [264]:
def index_split(train_ratio, valid_ratio, test_ratio, total_num = 300):
    if train_ratio + valid_ratio + test_ratio != 1:
        raise ValueError("Three input ratio should sum to be 1!")
    total_index = np.arange(1, total_num)
    rng = np.random.default_rng()
    total_index = rng.permutation(total_index)
    knot_1 = int(total_num * train_ratio)
    knot_2 = int(total_num * valid_ratio) + knot_1
    train_index, valid_index, test_index = np.split(total_index, [knot_1, knot_2])
    return train_index, valid_index, test_index

In [291]:
print(np.shape(fpcmesh.edges))

(2000, 2)


In [299]:
def loadfpc(index):
  vels = open("./csv_data/data_"+ str(index) + ".csv")
  velocities = np.loadtxt(vels, delimiter=",")[:,3:5]
  EdgeFeatures = np.array([[(velocities[edge[0]][0]+velocities[edge[1]][0])/2,(velocities[edge[0]][1]+velocities[edge[1]][1])/2] for edge in fpcmesh.edges])
  velx = torch.Tensor([[EdgeFeatures[:,0],EdgeFeatures[:,1]]])
  return velx

In [300]:
print(loadfpc(4).size())

torch.Size([1, 2, 10421])


In [294]:
def load_tensor(simulation_indexes):
    total = len(simulation_indexes)
    cnt_progress = 0
    bar=progressbar.ProgressBar(maxval=total)
    tensor = loadfpc(0)
    cnt_progress+=1
    bar.update(cnt_progress)    
    for i in range(1, total):
        tensor = torch.cat((tensor, loadfpc(i)))
        cnt_progress+=1
        bar.update(cnt_progress)
    bar.finish()
    return tensor

In [295]:
import progressbar


In [296]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1
train_index, valid_index, test_index = index_split(train_ratio, valid_ratio, test_ratio, total_num = 1000)

In [301]:
train_set = load_tensor(train_index)
valid_set = load_tensor(valid_index)
test_set = load_tensor(test_index)

100% (800 of 800) |######################| Elapsed Time: 0:01:16 Time:  0:01:16
100% (100 of 100) |######################| Elapsed Time: 0:00:08 Time:  0:00:08
100% (99 of 99) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


In [280]:
os.listdir()

['simple_hilbert.py',
 'advection_block.py',
 '__pycache__',
 'CAE_on_structured_mesh.ipynb',
 'Flowpast',
 'animation.mp4',
 'MeshCNN',
 'fpc.obj',
 'cache',
 'fpc_000.npz',
 'fpc.edges',
 'fpc_000.npz (Unzipped Files)',
 'Advection_of_a_block_over_structured_mesh.ipynb',
 'adobe.obj',
 'fpc2.obj',
 'Diss.ipynb',
 'csv_data',
 'fpc3.obj',
 'animation2.mp4',
 'Diss2.ipynb',
 'TENS']

In [302]:
torch.save(train_set, './TENS/train_tensor.pt')
torch.save(valid_set, './TENS/valid_tensor.pt')
torch.save(test_set, './TENS/test_tensor.pt')

In [303]:
train_set = torch.load('./TENS/train_tensor.pt', map_location=torch.device('cpu'))
valid_set = torch.load('./TENS/valid_tensor.pt', map_location=torch.device('cpu'))
test_set = torch.load('./TENS/test_tensor.pt', map_location=torch.device('cpu'))

In [304]:
print(train_set.size())

torch.Size([800, 2, 10421])


In [305]:
def train(autoencoder, optimizer, criterion, dataloader):
  autoencoder.train()
  train_loss, data_length = 0, len(dataloader.dataset)
  for batch in dataloader:
      batch = batch.to(device)  # Send batch of images to the GPU
      optimizer.zero_grad()  # Set optimiser grad to 0
      x_hat = autoencoder(batch,[fpcmesh])  # Generate predicted images (x_hat) by running batch of images through autoencoder
      MSE = criterion(batch, x_hat)  # Calculate MSE loss
      MSE.backward()  # Back-propagate
      optimizer.step()  # Step the optimiser
      train_loss += MSE * batch.size(0)

  return train_loss / data_length  # Return MSE

def validate(autoencoder, optimizer, criterion, dataloader):
    autoencoder.eval()
    validation_loss, data_length = 0, len(dataloader.dataset)
    for batch in dataloader:
        with torch.no_grad():
            batch = batch.to(device)  # Send batch of images to the GPU
            x_hat = autoencoder(batch)  # Generate predicted images (x_hat) by running batch of images through autoencoder
            MSE = criterion(batch, x_hat)  # Calculate MSE loss
            validation_loss += MSE * batch.size(0)

    return validation_loss / data_length   # Return MSE

In [2]:
def train_model(autoencoder, batch_size=16, n_epochs = 10, weight_decay = 1e-5, visualize=True):
  set_seed(42)
  autoencoder = autoencoder.to(device)
  optimizer = torch.optim.Adam(autoencoder.parameters())

  # we choose the MSE to be the loss function 
  criterion = nn.MSELoss()

  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
  valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
  
  # do livelossplot if visualize turned-on
  if visualize:
      liveloss = PlotLosses()

  for epoch in range(n_epochs):
    train_MSE = train(autoencoder, optimizer, criterion, train_loader)
    validation_MSE = validate(autoencoder, optimizer, criterion, valid_loader)
    print("eppoch %d starting......"%(epoch))
    
    # do livelossplot if visualize turned-on 
    if visualize: 
      logs = {}

      logs['' + 'log loss'] = train_MSE.item()
      logs['val_' + 'log loss'] = validation_MSE.item()

      liveloss.update(logs)
      liveloss.draw()

      
  return autoencoder

In [3]:
autoencoder = train_model(encdek)

NameError: ignored